# 🧪 NerdBud – Quiz Engine

This notebook handles:
- Loading quiz questions
- Displaying MCQs
- Capturing answers
- Tracking time and correctness


In [1]:
import json
import os
import time
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime


In [2]:
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
QUIZ_PATH = os.path.join(BASE_PATH, "data", "quizzes", "python_basics.json")
LOG_DIR = os.path.join(BASE_PATH, "data", "sample_logs")
ATTEMPT_LOG = os.path.join(LOG_DIR, "quiz_attempts.csv")

os.makedirs(LOG_DIR, exist_ok=True)

with open(QUIZ_PATH, "r") as f:
    quizzes = json.load(f)


In [3]:
current_question = 0
score = 0
start_time = None
selected_level = "Beginner"
responses = []


In [4]:
# Persistent output area for rendering questions and results
output = widgets.Output()
display(output)

Output()

In [5]:
def display_question():
    global start_time

    output.clear_output()

    q = quizzes[selected_level][current_question]

    question_label = widgets.HTML(
        f"<h4>Q{current_question + 1}: {q['question']}</h4>"
    )

    options = widgets.RadioButtons(
        options=q["options"]
    )

    submit_btn = widgets.Button(
        description="Submit",
        button_style="primary"
    )

    with output:
        display(question_label, options, submit_btn)

    start_time = time.time()

    def submit_answer(b):
        global score, current_question

        time_taken = round(time.time() - start_time, 2)
        correct = options.value == q["answer"]

        if correct:
            score += 1

        responses.append({
            "question_id": q["id"],
            "topic": q["topic"],
            "selected": options.value,
            "correct": correct,
            "time_taken": time_taken,
            "timestamp": datetime.now()
        })

        current_question += 1

        if current_question < len(quizzes[selected_level]):
            display_question()
        else:
            show_results()

    submit_btn.on_click(submit_answer)


In [6]:
def show_results():
    output.clear_output()

    total = len(quizzes[selected_level])
    accuracy = round(score / total, 2)

    with output:
        display("Quiz Completed ✅")
        display(f"Score: {score}/{total}")
        display(f"Accuracy: {accuracy * 100}%")

        df = pd.DataFrame(responses)
        df.to_csv(
            ATTEMPT_LOG,
            mode="a",
            header=not os.path.exists(ATTEMPT_LOG),
            index=False
        )

        display("Results saved successfully 📁")


In [7]:
start_quiz_btn = widgets.Button(
    description="Start Beginner Quiz",
    button_style="success"
)

def start_quiz(b):
    global current_question, score, responses
    current_question = 0
    score = 0
    responses = []
    display_question()

start_quiz_btn.on_click(start_quiz)
display(start_quiz_btn)


Button(button_style='success', description='Start Beginner Quiz', style=ButtonStyle())